# Connect to Label Studio

In [1]:
import os
from label_studio_sdk import LabelStudio

ls = LabelStudio(
    base_url=os.getenv('LABEL_STUDIO_URL', 'http://localhost:8080'),
    api_key=os.getenv('LABEL_STUDIO_API_KEY'),
)

# Create New Project

First we need to define a **Labeling Interface** for the new project:

In [2]:
from label_studio_sdk.label_interface import LabelInterface
from label_studio_sdk.label_interface.create import choices

label_config = LabelInterface.create({
    'text': 'Text',
    'label': choices(['Positive', 'Negative'])
})
print(label_config)

<View>
  <Text name="text" value="$text"/>
  <Choices name="label" toName="text">
    <Choice value="Positive"/>
    <Choice value="Negative"/>
  </Choices>
</View>



You can manually create [XML configuration for the labeling interface](https://labelstud.io/tags), but `LabelInterface` class provides a convenient way to create it programmatically.

Now let's create a new project with the specified labeling interface:

In [3]:
project = ls.projects.create(
    title='Text Classification',
    label_config=label_config
)

print(f'Created project:\n{ls._client_wrapper.get_base_url()}/projects/{project.id}')

Created project:
https://develop.appx.humansignal.com/projects/462621


# Import Tasks to the Project

In [4]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world"},
        {"text": "Hello Label Studio"},
        {"text": "What a beautiful day"},
    ]
)

ProjectsImportTasksResponse(annotation_count=None, could_be_tasks_list=None, data_columns=None, duration=None, file_upload_ids=None, found_formats=None, predictions_count=None, task_count=None, import=434811)

## Import Tasks with Pre-annotations

In [5]:
ls.projects.import_tasks(
    id=project.id,
    request=[
        {"text": "Hello world", "sentiment": "Positive"},
        {"text": "Goodbye Label Studio", "sentiment": "Negative"},
        {"text": "What a beautiful day", "sentiment": "Positive"},
    ],
    preannotated_from_fields=['sentiment']
)

ProjectsImportTasksResponse(annotation_count=None, could_be_tasks_list=None, data_columns=None, duration=None, file_upload_ids=None, found_formats=None, predictions_count=None, task_count=None, import=434814)

More customizable way to import preannotations:

In [6]:
from label_studio_sdk.label_interface.objects import PredictionValue

# this returns the same `LabelInterface` object as above
li = ls.projects.get(id=project.id).get_label_interface()

# by specifying what fields to `include` we can speed up task loading
for task in ls.tasks.list(project=project.id, include=["id"]):
    task_id = task.id
    prediction = PredictionValue(
        # tag predictions with specific model version string
        # it can help managing multiple models in Label Studio UI
        model_version='my_model_v1',
        # define your labels here
        result=[
            li.get_control('label').label(['Positive']),
        ]
    )
    ls.predictions.create(task=task_id, **prediction.model_dump())


# Export Annotations

In [7]:
for task in ls.tasks.list(project=project.id):
    print(task.id)
    print(task.annotations)
    print(task.predictions)

9079561
[]
[LseTaskPredictionsItem(created_at=datetime.datetime(2025, 10, 1, 21, 0, 54, 306773, tzinfo=datetime.timezone.utc), model=None, model_run=None, model_version='my_model_v1', project=462621, result=[{'id': '787be9ad-a2a7-411a-b567-ceeb84a5923a', 'type': 'choices', 'value': {'choices': ['Positive']}, 'to_name': 'text', 'from_name': 'label'}], score=0.0, task=9079561, updated_at=datetime.datetime(2025, 10, 1, 21, 0, 54, 306787, tzinfo=datetime.timezone.utc), id=3584252, created_ago='0\xa0minutes', cluster=None, neighbors=None, mislabeling=0.0)]
9079562
[]
[LseTaskPredictionsItem(created_at=datetime.datetime(2025, 10, 1, 21, 0, 54, 613908, tzinfo=datetime.timezone.utc), model=None, model_run=None, model_version='my_model_v1', project=462621, result=[{'id': '19f8fc4b-d2be-4700-9b8b-c172d130215b', 'type': 'choices', 'value': {'choices': ['Positive']}, 'to_name': 'text', 'from_name': 'label'}], score=0.0, task=9079562, updated_at=datetime.datetime(2025, 10, 1, 21, 0, 54, 613919, tzi

Read more about task, annotations and predictions format in Label Studio [documentation](https://labelstud.io/guide/export#Label-Studio-JSON-format-of-annotated-tasks).